# Housing Value and Unemployment: Is there a Correlation?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from src.cleaning import DataCleaner

In [ ]:
# initialize data cleaner class, insert file paths to raw data sets
data_cleaner = DataCleaner('data/raw/hpi_po_metro.csv', 'data/raw/ssamatab1.csv')

# clean unemployment data set
clean_unemployment_data = data_cleaner.clean_unemployment_data

# clean housing data set
clean_housing_data = data_cleaner.clean_housing_data

In [ ]:
# group clean_housing_data and clean_unemployment_data by year to allow the data to be joined seamlessly
grouped_housing_data = clean_housing_data.drop(columns=['Quarter','Not_Seasonally_Adjusted_Index'] ).groupby(['CBSA', 'Metro_Name', 'Year']).agg('mean').reset_index()
grouped_unemployment_data = clean_unemployment_data.drop(columns="Month").groupby(['LAUS_Code','State_FIPS_Code','FIPS_Code','Area','Year']).agg('mean').round(2).reset_index()

# move the decimal on the Unemployment_Rate column
grouped_unemployment_data["Unemployment_Rate"] = grouped_unemployment_data["Unemployment_Rate"] / 10

In [ ]:
# make SQL database with tables "Unemployment" and "Housing_value"
with sqlite3.connect("database.db") as conn:
    grouped_unemployment_data.to_sql(
        "Unemployment", 
        conn, 
        if_exists='replace', 
        index=False
        )
    grouped_housing_data.to_sql(
        "Housing_value", 
        conn, 
        if_exists='replace', 
        index=False
        )

In [ ]:
# SQL query that joins the two tables Unemployment and Housing_value on the CBSA/FIPS code 
combined_tables_query = """

SELECT 
    Unemployment.LAUS_Code,
    Unemployment.Year,
    Unemployment.Area,
    Unemployment.Employment,
    Unemployment.Unemployment,
    Unemployment.Unemployment_Rate,
    Housing_value.Metro_Name,
    Housing_value.Seasonally_Adjusted_Index
FROM Unemployment
INNER JOIN Housing_value
    ON Unemployment.FIPS_Code = Housing_value.CBSA
    AND Unemployment.Year = Housing_value.Year
    
"""

combined_tables_result = pd.read_sql(combined_tables_query, conn)

In [ ]:
# group the combined table by area, returning the metro name, unemployment rate, and seasonally adjusted index.
combined_tables_result_grouped_by_area = combined_tables_result.groupby(["Metro_Name"])[["Unemployment_Rate", "Seasonally_Adjusted_Index"]].mean().round(2).reset_index()

# find the correlation by area
correlations_table = combined_tables_result.groupby(["Metro_Name"]).apply(
        lambda x: x["Unemployment_Rate"].corr(x["Seasonally_Adjusted_Index"]).round(2)
).rename("Correlation").reset_index()

In [ ]:
# insert these two dataframes into database.db
with sqlite3.connect("database.db") as conn:
    combined_tables_result_grouped_by_area.to_sql(
        name="Unemployment_Rate_and_SAI_by_Metro_Area",
        con=conn, 
        if_exists="replace"
        )
    correlations_table.to_sql(
        name="Correlations_by_Metro_Area",
        con=conn,
        if_exists="replace"
        )

In [ ]:
# SQL query to merge the tables "Unemployment_Rate_and_SAI_by_Metro_Area and Correlations_by_Metro_Area."
merged_correlations_table_query = """

SELECT
    Unemployment_Rate_and_SAI_by_Metro_Area.Metro_Name,
    Unemployment_Rate_and_SAI_by_Metro_Area.Unemployment_Rate,
    Unemployment_Rate_and_SAI_by_Metro_Area.Seasonally_Adjusted_Index,
    Correlations_by_Metro_Area.Correlation
FROM Unemployment_Rate_and_SAI_by_Metro_Area
LEFT JOIN Correlations_by_Metro_Area
    ON Unemployment_Rate_and_SAI_by_Metro_Area.Metro_Name = Correlations_by_Metro_Area.Metro_Name
ORDER BY Correlations_by_Metro_Area.Correlation DESC

"""
merged_correlations_table = pd.read_sql(merged_correlations_table_query, conn)

# Correlation Across the United States

In [ ]:
x = combined_tables_result['Unemployment_Rate']
y = combined_tables_result['Seasonally_Adjusted_Index']

correlation = x.corr(y).round(2)
slope, intercept = np.polyfit(x, y, 1)
y_fit = (slope * x) + intercept

fig, ax = plt.subplots(figsize=(12,8))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Unemployment Rate (%)", fontweight='bold', fontsize=16)
ax.set_ylabel("Housing Value Index Seasonally Adjusted",fontweight='bold', fontsize=16)
ax.set_title("National Correlation Between Unemployment and Housing Value", fontweight='bold', fontsize=16)
ax.grid(True, linestyle='--', alpha=0.5)
ax.plot(x, y_fit, color="darkviolet", linewidth=2.5, linestyle="--")
ax.text(
    x.max() * 0.80,
    y.max() * 0.90, 
    f'Correlation = {correlation}',
    fontweight="bold", 
    fontsize=12, 
    bbox={'facecolor': 'white', 'edgecolor': 'darkviolet'}
    )

sc = ax.scatter(x,y)
plt.tight_layout()
plt.show()

# Is the Negative Correlation Stronger in Certain Metro Areas?

In [ ]:
x = merged_correlations_table["Metro_Name"]
y = merged_correlations_table["Correlation"]

fig, ax = plt.subplots(figsize=(12,12))

# set the title, spine visibility, and axis labels
ax.set_title("Negative Correlation by Metro Area", fontweight="bold", fontsize=14)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Negative Correlation", fontweight="bold", fontsize=12)
ax.set_ylabel("Metro Area", fontweight="bold", fontsize=12)
ax.set_xlim(0 , -1)

# plot as a horizontal bar chart
bars = plt.barh(x,y)

# Used a for loop to get the correlation values and place them just to the right of the bar. 
for bar in bars:
    value = bar.get_width()
    ax.text(x=value - 0.01, y=(bar.get_y() + bar.get_height() - 0.50),s=f'{value:.2f}', ha='left', va='center')

plt.tight_layout()
plt.show()